In [ ]:
# Core libraries
import os
import re
import time
import glob
import pickle

# Data manipulation
import pandas as pd
import numpy as np

# Linear algebra utilities
from numpy.linalg import norm

# Text preprocessing
import unidecode
import nltk
# nltk.download('stopwords')  # Uncomment this if stopwords are not downloaded yet
from nltk.corpus import stopwords

# Parquet file handling
import pyarrow.parquet as pq

# Machine learning and embeddings
from sentence_transformers import SentenceTransformer, SimilarityFunction
import torch

# Progress bar utility
from tqdm import tqdm

# OpenAI API (if needed later in your pipeline)
import openai

In [ ]:
# Load the cleaned OpenStreetMap (OSM) categories dataset
df_osm = pd.read_csv('Database/Clean_categories/categories_OSM_clean.csv', sep=';')

# Load the cleaned Foursquare (FS) categories dataset
df_fs = pd.read_csv('Database/Clean_categories/categories_FS_clean.csv', sep=';')

# Load the Foursquare categories dataset with additional textual descriptions
df_fs_desc = pd.read_csv('Database/Clean_categories/categories_FS_clean_description.csv', sep=';')

# Load the "oracle" dataset, probably used as ground truth or reference mapping
df_oracle = pd.read_csv('df_oracle.csv', sep=";")

# Cleaning

In [ ]:
# Define a set of English stopwords from NLTK
stop_words = set(stopwords.words('english'))

# Function to clean text fields
def clean_text(text):
    if text is None or pd.isna(text):  # keep None or NaN values unchanged
        return None
    
    text = str(text).lower()                        # convert to lowercase
    text = unidecode.unidecode(text)                # remove accents/diacritics
    text = re.sub(r'[^\w\s]', '', text)             # remove punctuation
    words = text.split()                            # split into individual words
    words = [word for word in words if word not in stop_words]  # remove stopwords
    return ' '.join(words)                          # join words back into a string

# Apply text cleaning to OSM dataset
df_oracle['OSM_tag'] = df_oracle['OSM_tag'].apply(clean_text)
df_oracle['FS_tag'] = df_oracle['FS_tag'].apply(clean_text)

df_osm['Tag'] = df_osm['Tag'].apply(clean_text)
df_osm['token_Description'] = df_osm['Description'].apply(clean_text)
df_osm['Depth_1'] = df_osm['Depth_1'].apply(clean_text)
df_osm['Depth_2'] = df_osm['Depth_2'].apply(clean_text)
df_osm['Depth_3'] = df_osm['Depth_3'].apply(clean_text)

# Apply text cleaning to Foursquare dataset
df_fs['Tag'] = df_fs['Tag'].apply(clean_text)
df_fs['Depth_1'] = df_fs['Depth_1'].apply(clean_text)
df_fs['Depth_2'] = df_fs['Depth_2'].apply(clean_text)
df_fs['Depth_3'] = df_fs['Depth_3'].apply(clean_text)
df_fs['Depth_4'] = df_fs['Depth_4'].apply(clean_text)
df_fs['Depth_5'] = df_fs['Depth_5'].apply(clean_text)
df_fs['Depth_6'] = df_fs['Depth_6'].apply(clean_text)

# Apply text cleaning to Foursquare dataset with descriptions
df_fs_desc['Tag'] = df_fs_desc['Tag'].apply(clean_text)
df_fs_desc['Depth_1'] = df_fs_desc['Depth_1'].apply(clean_text)
df_fs_desc['Depth_2'] = df_fs_desc['Depth_2'].apply(clean_text)
df_fs_desc['Depth_3'] = df_fs_desc['Depth_3'].apply(clean_text)
df_fs_desc['Depth_4'] = df_fs_desc['Depth_4'].apply(clean_text)
df_fs_desc['Depth_5'] = df_fs_desc['Depth_5'].apply(clean_text)
df_fs_desc['Depth_6'] = df_fs_desc['Depth_6'].apply(clean_text)
df_fs_desc = df_fs_desc.rename(columns={"definition_en": "Description"})  # rename column for clarity
df_fs_desc['token_Description'] = df_fs_desc['Description'].apply(clean_text)

# Create cleaned versions of the datasets for later use
df_osm_clean = df_osm
df_fs_clean = df_fs
df_fs_desc_clean = df_fs_desc

In [ ]:
# we create in each dataset a column that contains all the information about the POI
def concat_columns_osm(row):
    parts = []
    for col in ['Depth_1', 'Depth_2', 'Depth_3', 'token_Description']:
        val = row[col]
        if val is not None and pd.notna(val):  # Ignore None and NaN values
            parts.append(str(val))
    return ' '.join(parts)

def concat_columns_fs(row):
    parts = []
    for col in ['Depth_1', 'Depth_2', 'Depth_3', 'Depth_4', "Depth_5", "Depth_6"]:
        val = row[col]
        if val is not None and pd.notna(val):  # Ignore None and NaN values
            parts.append(str(val))
    return ' '.join(parts)

def concat_columns_fs_desc(row):
    parts = []
    for col in ['Depth_1', 'Depth_2', 'Depth_3', 'Depth_4', "Depth_5", "Depth_6", "token_Description"]:
        val = row[col]
        if val is not None and pd.notna(val):  # Ignore None and NaN values
            parts.append(str(val))
    return ' '.join(parts)


# Create new columns by concatenating multiple text fields into a single string
df_osm['full_info'] = df_osm.apply(concat_columns_osm, axis=1)
df_fs_desc['full_info_and_desc'] = df_fs_desc.apply(concat_columns_fs_desc, axis=1)
df_fs_desc['full_info'] = df_fs_desc.apply(concat_columns_fs, axis=1)



# we create a column in each dataset that contains the path of the POI in the categorization
def concat_depths_osm(row):
    levels = []
    for col in ['Depth_1', 'Depth_2', 'Depth_3']:
        val = row[col]
        if pd.notna(val) and val is not None and str(val).strip() != '':
            levels.append(str(val).strip())
    return ' > '.join(levels)

df_osm['Path'] = df_osm.apply(concat_depths_osm, axis=1)

def concat_depths_fs(row):
    levels = []
    for col in ['Depth_1', 'Depth_2', 'Depth_3', 'Depth_4', "Depth_5", "Depth_6"]:
        val = row[col]
        if pd.notna(val) and val is not None and str(val).strip() != '':
            levels.append(str(val).strip())
    return ' > '.join(levels)

#df_fs['Path'] = df_fs.apply(concat_depths_fs, axis=1)  # optional, commented out
df_fs_desc['Path'] = df_fs_desc.apply(concat_depths_fs, axis=1)



# Remove duplicate words while keeping order (convert to dict keys then join)
df_fs_desc["full_info_and_desc_set"] = df_fs_desc["full_info_and_desc"].apply(
    lambda x: " ".join(dict.fromkeys(x.split()))
)

df_osm["full_info_set"] = df_osm["full_info"].apply(
    lambda x: " ".join(dict.fromkeys(x.split()))
)

# ChatGPT as a final chooser

In [ ]:
import pandas as pd
stop_words = set(stopwords.words('english'))

# Function to clean text fields
def clean_text(text):
    if text is None or pd.isna(text):  # keep None or NaN as they are
        return None
    
    text = str(text).lower()                        # convert to lowercase
    text = unidecode.unidecode(text)                # remove accents
    text = re.sub(r'[^\w\s]', '', text)             # remove punctuation
    words = text.split()                            # split into words
    words = [word for word in words if word not in stop_words]  # remove stopwords
    return ' '.join(words)                          # rejoin into cleaned string

# Apply text cleaning to Foursquare dataset columns
df_fs['Tag'] = df_fs['Tag'].apply(clean_text)
df_fs['Depth_1'] = df_fs['Depth_1'].apply(clean_text)
df_fs['Depth_2'] = df_fs['Depth_2'].apply(clean_text)
df_fs['Depth_3'] = df_fs['Depth_3'].apply(clean_text)
df_fs['Depth_4'] = df_fs['Depth_4'].apply(clean_text)
df_fs['Depth_5'] = df_fs['Depth_5'].apply(clean_text)
df_fs['Depth_6'] = df_fs['Depth_6'].apply(clean_text)

# Function to concatenate hierarchical depth columns into a single path string
def concat_depths_fs(row):
    levels = []
    for col in ['Depth_1', 'Depth_2', 'Depth_3', 'Depth_4', "Depth_5", "Depth_6"]:
        val = row[col]
        if pd.notna(val) and val is not None and str(val).strip() != '':
            levels.append(str(val).strip())
    return ' > '.join(levels)

# Add a new column with the full category path of each tag
df_fs['Path'] = df_fs.apply(concat_depths_fs, axis=1)

# Build a dictionary mapping: tag -> full path
tag_to_path = dict(zip(df_fs["Tag"], df_fs["Path"]))

# Build a dictionary mapping: tag -> main category (Depth_1)
tag_to_main = dict(zip(df_fs["Tag"], df_fs["Depth_1"]))

## Embeddings

In [ ]:
client = openai.OpenAI(api_key="your_api_key")

model = SentenceTransformer('all-MiniLM-L6-v2')  # fast model

def make_matchs(model, model_name, description_osm, description_fs, df_osm, df_fs_desc, desc_fs="", k=5):
    model.similarity_fn_name = SimilarityFunction.COSINE  
    
    # Compute embeddings for OSM descriptions
    print("-- beginning embedding OSM")
    embeddings_1 = model.encode(description_osm, convert_to_tensor=True) 
    print("-- end embedding OSM")

    # Compute embeddings for FS descriptions
    print("-- beginning embedding FS")
    embeddings_2 = model.encode(description_fs, convert_to_tensor=True) 
    print("-- end embedding FS")

    # Cosine similarity between OSM and FS embeddings
    similarity_score = model.similarity(embeddings_1, embeddings_2)

    matches = []

    # Iterate through OSM categories
    for idx1, row1 in df_osm.iterrows():
        # Get top-k indices and similarity scores for each OSM entry
        topk_scores, topk_indices = torch.topk(similarity_score[idx1], k)

        # Extract FS tags and corresponding scores
        topk_tags = df_fs_desc.iloc[topk_indices.tolist()]['Tag'].tolist()
        topk_scores = topk_scores.tolist()
        
        # Store match information
        matches.append({
            'OSM_tag': row1['Tag'],
            'OSM_description': row1['Description'],
            'OSM_main': row1["Depth_1"],
            'OSM_path': row1['Path'],
            'FS_topk_tags': topk_tags,
            'FS_topk_scores': topk_scores
        })

    # Helper to get FS paths for top-k tags
    def get_paths(topk_tags):
            return [tag_to_path.get(tag, None) for tag in topk_tags]
    
    # Helper to get FS main categories for top-k tags
    def get_main(topk_tags):
            return [tag_to_main.get(tag, None) for tag in topk_tags]

    # Convert matches to DataFrame and enrich with paths and main categories
    matches_df = pd.DataFrame(matches)
    matches_df['FS_topk_paths'] = matches_df['FS_topk_tags'].apply(get_paths)
    matches_df["FS_topk_main"] = matches_df["FS_topk_tags"].apply(get_main)
    
    return matches_df

## Different ChatGPT prompts

In [15]:
def ask_gpt_to_choose_prompt_2(osm_tag, osm_path, osm_desc, candidates, k):
    # Construire le prompt avec règles supplémentaires
    prompt = f"""
    I want to map an OpenStreetMap (OSM) point of interest (POI) to the most appropriate FourSquare (FS) POI (tag).

    Rules:
    1. If an FS tag exactly matches the OSM tag (same name or clear synonym), choose that FS tag directly.
    2. If no exact match exists, choose the FS tag that is the most specific and precise category in which the OSM tag could reasonably be classified.
    - Do not just pick the most semantically similar.
    - Prefer the FS tag that fully contains the concept of the OSM tag, even if its wording is broader.
    3. Always answer with exactly one FS tag name from the provided list, nothing else.

    OSM Tag: {osm_tag}  
    OSM Tag Description: {osm_desc}  
    OSM Tag Categorisation in OSM: {osm_path}  

    Here are the {k} most relevant FS tags (with their descriptions):  
    {candidates.to_string(index=False)}  

    Question: Which FS tag best matches the OSM tag?  
    Answer only with the FS tag name. """ 


    response = client.chat.completions.create(
        model="gpt-4o-mini",  # rapide et pas cher
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    return response.choices[0].message.content.strip()



In [ ]:
def ask_gpt_to_choose_prompt_3(osm_tag, osm_path, osm_desc, candidates, k):
    # Construire le prompt avec règles supplémentaires
    prompt = f"""
    I want to map an OpenStreetMap (OSM) point of interest (POI) to the most appropriate FourSquare (FS) POI (tag).

    Rules:
    1. If an FS tag exactly matches the OSM tag (same name or clear synonym), choose that FS tag directly.
    2. If no exact match exists, choose the FS tag that is the most specific and precise category in which the OSM tag could reasonably be classified.
    - Do not just pick the most semantically similar.
    - Prefer the FS tag that fully contains the concept of the OSM tag, even if its wording is broader.
    3. If none of the provided FS tags are a good fit, you may instead choose one category from the following broader fallback categories:
    - landmarks outdoors
    - business professional services
    - travel transportation
    - community government
    - retail
    - sports recreation
    - health medicine
    - arts entertainment
    - dining drinking
    - event
    4. Always answer with exactly one FS tag name from the provided list or, if necessary, one fallback category, nothing else.


    OSM Tag: {osm_tag}  
    OSM Tag Description: {osm_desc}  
    OSM Tag Categorisation in OSM: {osm_path}  

    Here are the {k} most relevant FS tags (with their descriptions):  
    {candidates.to_string(index=False)}  

    Question: Which FS tag best matches the OSM tag?  
    Answer only with the FS tag name. """ 


    response = client.chat.completions.create(
        model="gpt-4o-mini",  #rapide et pas cher
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    return response.choices[0].message.content.strip()



In [17]:
def ask_gpt_to_choose_prompt_5(osm_tag, osm_path, osm_desc, candidates, k):
    prompt = f"""
    I want to map an OpenStreetMap (OSM) point of interest (POI) to the most appropriate FourSquare (FS) POI (tag).

    Rules:
    1. If an FS tag exactly matches the OSM tag (same name or clear synonym), choose that FS tag directly.
    2. If no exact match exists, select the FS tag that is the most precise parent category that fully contains the OSM tag.
       - Do not just pick the most semantically similar wording.
       - Prefer a broader FS tag that logically includes the OSM tag concept, even if it is less specific.
       - Exclude FS tags that are related but do not actually contain the OSM concept.
    3. Think step by step: first check for an exact match, then find the correct parent category.
    4. Output only one FS tag name from the provided list. No explanations, no extra text.

    Example:
    OSM Tag: "sea"  
    OSM Tag Description: "A large body of salt water part of, or connected to, an ocean."  
    OSM Tag Categorisation in OSM: "place > sea"  

    FS Candidates:  
    ['lake', 'bay', 'bathing area', 'dive spot', 'island', 'surf spot', 'waterfront', 'river', 'landmarks outdoors', 'boat launch']

    Correct Answer: landmarks outdoors

    Now, process the following case:

    OSM Tag: {osm_tag}  
    OSM Tag Description: {osm_desc}  
    OSM Tag Categorisation in OSM: {osm_path}  

    Here are the {k} most relevant FS tags (with their descriptions):  
    {candidates.to_string(index=False)}  

    Question: Which FS tag best matches the OSM tag?  
    Answer only with the FS tag name.
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",  # rapide et pas cher
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    return response.choices[0].message.content.strip()


In [11]:
def ask_gpt_to_choose_prompt_6(osm_tag, osm_path, osm_desc, candidates, k):
    prompt = f"""
    I want to map an OpenStreetMap (OSM) point of interest (POI) to the most appropriate FourSquare (FS) POI (tag).

    Rules:
    1. If an FS tag exactly matches the OSM tag (same name or clear synonym), choose that FS tag directly.
    2. If no exact match exists, select the FS tag that is the most precise parent category that fully contains the OSM tag.
       - Do not just pick the most semantically similar wording.
       - Prefer a broader FS tag that logically includes the OSM tag concept.
       - Exclude FS tags that are related but do not actually contain the OSM concept.
    3. If none of the {k} FS candidates are suitable, then choose from the following broader FS categories:
       - landmarks outdoors
       - business professional services
       - travel transportation
       - community government
       - retail
       - sports recreation
       - health medicine
       - arts entertainment
       - dining drinking
       - event
    4. Think step by step: first check for an exact match, then find the correct parent category, and only if needed, fall back to the broader FS categories above.
    5. Output only one FS tag name from the provided list (either from the {k} candidates or from the broader categories).
       Do not add explanations, reasoning, or extra text.

    Example:
    OSM Tag: "sea"  
    OSM Tag Description: "A large body of salt water part of, or connected to, an ocean."  
    OSM Tag Categorisation in OSM: "place > sea"  

    FS Candidates:  
    ['lake', 'bay', 'bathing area', 'dive spot', 'island', 'surf spot', 'waterfront', 'river', 'landmarks outdoors', 'boat launch']

    Correct Answer: landmarks outdoors 

    Now, process the following case:

    OSM Tag: {osm_tag}  
    OSM Tag Description: {osm_desc}  
    OSM Tag Categorisation in OSM: {osm_path}  

    Here are the {k} most relevant FS tags (with their descriptions):  
    {candidates.to_string(index=False)}  

    Question: Which FS tag best matches the OSM tag?  
    Answer only with the FS tag name.
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",  # rapide et pas cher
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    return response.choices[0].message.content.strip()

## Choice by CHATGPT

In [ ]:
import pandas as pd
from tqdm import tqdm

# Select the prompts we want to test
prompts = [2,3,5,6]
# Choose the value of k
k=20 # number of candidates to provide to ChatGPT

def concat_depths_fs(row):
    levels = []
    for col in ['Depth_1', 'Depth_2', 'Depth_3', 'Depth_4', "Depth_5", "Depth_6"]:
        val = row[col]
        if pd.notna(val) and val is not None and str(val).strip() != '':
            levels.append(str(val).strip())
    return ' > '.join(levels)

# Create a full path column by concatenating all depth levels
df_fs['Path'] = df_fs.apply(concat_depths_fs, axis=1)

# Build a dictionary mapping FS tag -> path
tag_to_path = dict(zip(df_fs["Tag"], df_fs["Path"]))

# Generate matches using the function make_matchs
matches_df = make_matchs(
    model=model,
    model_name="MiniLM",
    description_osm=df_osm["full_info"].tolist(),
    description_fs=df_fs_desc["full_info_and_desc"].tolist(),
    df_osm=df_osm,
    df_fs_desc=df_fs_desc,
    k=k  
)

# Loop through each selected prompt
for script in prompts:
    k = k # or another value if needed
    results = []

    print(f"--- Running prompt {script} ---")

    # Iterate through each row of the matches dataframe with a progress bar
    for idx, row in tqdm(matches_df.iterrows(), total=len(matches_df)):
        osm_tag = row["OSM_tag"]
        osm_desc = row["OSM_description"]
        osm_path = row["OSM_path"]
        fs_candidates = row["FS_topk_tags"]
        fs_path = row["FS_topk_paths"]
        fs_scores = row["FS_topk_scores"]
        fs_main = row["FS_topk_main"]

        # Temporary dataframe for the candidates of this prompt
        candidates = pd.DataFrame({
            "FS_tag": fs_candidates,
            "Score": fs_scores
        })

        # Call the GPT API according to the prompt
        if script == 2:
            fs_choice = ask_gpt_to_choose_prompt_2(osm_tag, osm_path, osm_desc, candidates, k)
        elif script == 3:
            fs_choice = ask_gpt_to_choose_prompt_3(osm_tag, osm_path, osm_desc, candidates, k)
        elif script == 5:
            fs_choice = ask_gpt_to_choose_prompt_5(osm_tag, osm_path, osm_desc, candidates, k)
        elif script == 6:
            fs_choice = ask_gpt_to_choose_prompt_6(osm_tag, osm_path, osm_desc, candidates, k)
        else :
            print("PROBLEEEEEEEEEEM")

        # Store the results for this row
        results.append({
            "OSM_tag": osm_tag,
            "OSM_path": osm_path,
            "OSM_description": osm_desc,
            "FS_candidates": fs_candidates,
            "FS_scores": fs_scores,
            "FS_tag_GPT": fs_choice
        })

    # Create the final dataframe for this prompt
    df_gpt_mapping = pd.DataFrame(results)

    # Merge with the oracle dataframe
    df_oracle = pd.read_csv('df_oracle.csv', sep=";")
    df_match = pd.merge(df_gpt_mapping, df_oracle, 
                        left_on=["OSM_tag", "OSM_path"], 
                        right_on=['OSM_tag', "OSM_path"], how="left")
    df_match = df_match.rename(columns={"FS_tag":"FS_manual_tag"})

    # Create a column indicating if the GPT prediction matches the manual FS tag
    df_match["correct"] = (df_match['FS_manual_tag'] == df_match['FS_tag_GPT']).astype(int)
        
    def is_new_correct(fs_manual_tag, fs_tag_gpt, tag_to_path):
        # Condition 1: exact match
        if fs_manual_tag == fs_tag_gpt:
            return 1
        # Condition 2: FS_manual_tag appears anywhere in the path of FS_tag_GPT
        path_str = tag_to_path.get(fs_tag_gpt, "")  # get path as string
        path_tags = [tag.strip() for tag in path_str.split(">")]  # split path into tags
        if fs_manual_tag in path_tags:
            return 1
        return 0

    # Apply the function row by row to create a new_correct column
    df_match["new_correct"] = df_match.apply(
        lambda row: is_new_correct(row["FS_manual_tag"], row["FS_tag_GPT"], tag_to_path),
        axis=1
    )

    # Extract the main OSM category and the first two levels
    df_match["OSM_main"] = df_match["OSM_path"].str.split(" > ").str[0]
    df_match["OSM_main_sub"] = df_match["OSM_path"].apply(lambda x: " > ".join(x.split(" > ")[:2]))

    # Save the dataframe immediately after each prompt
    filename = f"FID gpt models/miniLM + chat_gpt_prompt_{script} (k={k}).csv"
    df_match.to_csv(filename, index=False, sep=";")
    print(f"✅ Prompt {script} completed and saved to {filename}\n")